# Программирование для всех (основы Python)

*Алла Тамбовцева*

## Введение в парсинг с BeautifulSoup: поиск по тэгам

Импортируем необходимые библиотеки. Нам понадобится модуль `requests` для отправки запросов, для «подключения» к странице и получения её содержимого в виде строки, и функция `BeautifulSoup` из библиотеки `bs4` для удобного поиска по полученной строке:

In [1]:
import requests
from bs4 import BeautifulSoup

Модуль `requests` является базовым, его не нужно устанавливать отдельно, библиотека `bs4` таковой не является. Если вы не устанавливали Anaconda, скорее всего, на строку `from bs4 import BeautifulSoup` Python выдаст сообщение, что модуль не найден, поэтому его нужно будет установить (если ошибок при импорте нет, не запускайте следующую ячейку):

In [ ]:
!pip install bs4

Подключаемся к странице сайта с материалами [math-info.hse.ru](http://math-info.hse.ru):

In [2]:
page = requests.get("http://math-info.hse.ru/2023-24")

In [3]:
page

<Response [404]>

Объект, который мы сохранили в `page`, имеет особый тип *request*, он же запрос. Из него можно извлечь исходный код страницы в виде обычного текста (тип *string*). Однако в нашем случае мы этого сделать не сможем, потому что в ответ на запрос мы получили результат с кодом 404. Известная ошибка, которая возникает тогда, когда ссылка ведёт вникуда. Исправляемся – добавляем слэш в конце ссылки, в данном случае это важно:

In [4]:
page = requests.get("http://math-info.hse.ru/2023-24/")
page

<Response [200]>

Теперь код 200, это означает, что запрос был благополучно отправлен и на него был получен ответ. Извлечём из полученного ответа код HTML в виде обычной строки:

In [ ]:
# page.text

Выполнять поиск по такой строке с тэгами не очень удобно (даже если вы знакомы с регулярными выражениями), поэтому преобразуем строку в объект типа `BeautifulSoup`. Такой объект внешне несильно отличается от обычной строки, однако внутри по структуре похож на словарь, и это сходство значительно упрощает поиск по тэгам и атрибутам.

In [5]:
soup = BeautifulSoup(page.text)

In [ ]:
# внешне то же самое, что и ранее
# soup

На объектах `BeautifulSoup` определены методы `.find()` и `.find_all()`. Оба метода возвращают фрагменты кода HTML, которые соответствуют критериям поиска, различие заключается в том, что метод `.find()` предназначен для поиска одного совпадения (если критериям поиска соответствует несколько элементов на странице, то берётся только первый), а метод `.find_all()` – для поиска всех совпадений. В первом случае результат возвращается в виде одного элемента типа `BeautifulSoup`, а во втором – в виде списка таких элементов. Давайте попробуем что-то поискать!

Найдём сначала заголовок первого уровня – он здесь точно один (тэг `<h1>`):

In [6]:
soup.find("h1")

<h1 class="firstHeading" id="firstHeading" lang="ru">Заглавная страница</h1>

Результат – фрагмент кода HTML со всеми тэгами. Как получить «чистый» текст? Запросить текст с помощью атрибута `.text`:

In [7]:
soup.find("h1").text

'Заглавная страница'

Как забрать значение какого-нибудь атрибута выше, например, значение `ru` из `lang`? Воспользоваться тем, что содержимое тэга в `BeautifulSoup` по структуре похоже на словарь. То есть, содержимое тэга `<h1>` выше – это «доработанный» словарь вида

    {"class" : "firstHeading",
    "id" : "firstHeading",
    "lang" : "ru"}.
    
А записи из словаря мы умеем извлекать. Либо по ключу в квадратных скобках:

In [8]:
soup.find("h1")["lang"]

'ru'

Либо (более универсально, на случай, чтобы код не выдавал `KeyError`, если такого атрибута не существует) через метод `.get()`:

In [9]:
soup.find("h1").get("lang")

'ru'

Теперь найдём все заголовки второго уровня:

In [10]:
soup.find_all("h2")

[<h2>Содержание</h2>,
 <h2><span id="Школа_лингвистики"></span><span class="mw-headline" id=".D0.A8.D0.BA.D0.BE.D0.BB.D0.B0_.D0.BB.D0.B8.D0.BD.D0.B3.D0.B2.D0.B8.D1.81.D1.82.D0.B8.D0.BA.D0.B8">Школа лингвистики</span></h2>,
 <h2><span id="ОП_Политология"></span><span class="mw-headline" id=".D0.9E.D0.9F_.D0.9F.D0.BE.D0.BB.D0.B8.D1.82.D0.BE.D0.BB.D0.BE.D0.B3.D0.B8.D1.8F">ОП Политология</span></h2>,
 <h2><span id="Вычислительные_социальные_науки"></span><span class="mw-headline" id=".D0.92.D1.8B.D1.87.D0.B8.D1.81.D0.BB.D0.B8.D1.82.D0.B5.D0.BB.D1.8C.D0.BD.D1.8B.D0.B5_.D1.81.D0.BE.D1.86.D0.B8.D0.B0.D0.BB.D1.8C.D0.BD.D1.8B.D0.B5_.D0.BD.D0.B0.D1.83.D0.BA.D0.B8">Вычислительные социальные науки</span></h2>,
 <h2><span id="Факультет_креативных_индустрий"></span><span class="mw-headline" id=".D0.A4.D0.B0.D0.BA.D1.83.D0.BB.D1.8C.D1.82.D0.B5.D1.82_.D0.BA.D1.80.D0.B5.D0.B0.D1.82.D0.B8.D0.B2.D0.BD.D1.8B.D1.85_.D0.B8.D0.BD.D0.B4.D1.83.D1.81.D1.82.D1.80.D0.B8.D0.B9">Факультет креативных индустрий</span

Сохраним полученный список с названием `h2` и поработаем с ним (и не только)!

In [11]:
h2 = soup.find_all("h2")

### Задача 1

Сохраните в новый список `hh` только названия образовательных программ или факультетов/школ, то есть все элементы списка `h2`, кроме первого и последнего.

In [12]:
hh = h2[1:-1]
print(hh)

[<h2><span id="Школа_лингвистики"></span><span class="mw-headline" id=".D0.A8.D0.BA.D0.BE.D0.BB.D0.B0_.D0.BB.D0.B8.D0.BD.D0.B3.D0.B2.D0.B8.D1.81.D1.82.D0.B8.D0.BA.D0.B8">Школа лингвистики</span></h2>, <h2><span id="ОП_Политология"></span><span class="mw-headline" id=".D0.9E.D0.9F_.D0.9F.D0.BE.D0.BB.D0.B8.D1.82.D0.BE.D0.BB.D0.BE.D0.B3.D0.B8.D1.8F">ОП Политология</span></h2>, <h2><span id="Вычислительные_социальные_науки"></span><span class="mw-headline" id=".D0.92.D1.8B.D1.87.D0.B8.D1.81.D0.BB.D0.B8.D1.82.D0.B5.D0.BB.D1.8C.D0.BD.D1.8B.D0.B5_.D1.81.D0.BE.D1.86.D0.B8.D0.B0.D0.BB.D1.8C.D0.BD.D1.8B.D0.B5_.D0.BD.D0.B0.D1.83.D0.BA.D0.B8">Вычислительные социальные науки</span></h2>, <h2><span id="Факультет_креативных_индустрий"></span><span class="mw-headline" id=".D0.A4.D0.B0.D0.BA.D1.83.D0.BB.D1.8C.D1.82.D0.B5.D1.82_.D0.BA.D1.80.D0.B5.D0.B0.D1.82.D0.B8.D0.B2.D0.BD.D1.8B.D1.85_.D0.B8.D0.BD.D0.B4.D1.83.D1.81.D1.82.D1.80.D0.B8.D0.B9">Факультет креативных индустрий</span></h2>, <h2><span id="ОП_

### Задача 2

Извлеките из каждого элемента списка `hh` текст и сохраните полученные названия программ/факультетов/школ в список `headers`.

In [13]:
# с циклом
headers = []
for h in hh:
    headers.append(h.text)
print(headers)

['Школа лингвистики', 'ОП Политология', 'Вычислительные социальные науки', 'Факультет креативных индустрий', 'ОП социология', 'ОП Психология', 'МИЭФ', 'Разработка информационных систем для бизнеса (Пермь)']


In [14]:
# со списковым включением
headers = [h.text for h in hh]
print(headers)

['Школа лингвистики', 'ОП Политология', 'Вычислительные социальные науки', 'Факультет креативных индустрий', 'ОП социология', 'ОП Психология', 'МИЭФ', 'Разработка информационных систем для бизнеса (Пермь)']


### Задача 3

Найдите на странице список курсов ОП «Политология» (их просто больше всего) и сохраните его в переменную `polit`.

In [15]:
# находим все ненумерованные списки и берем третий
polit = soup.find_all("ul")[2]

### Задача 4

Используя `polit`, получите список названий курсов и назовите его `names`.

In [16]:
# элементы списка с тэгами
courses = polit.find_all("li")

# извлекаем текст из каждого элемента
names = [c.text for c in courses]
print(names)

['Высшая математика (1 курс)', 'Введение в регрессионный анализ (2 курс)', 'Основы анализа данных в Python (3 курс)', 'Программирование для всех (основы Python) (1 курс магистратуры)', 'Введение в теорию вероятностей и математическую статистику (1 курс)', 'Программирование для всех: продолжение (1 курс магистратуры)']


### Задача 5

Получите список ссылок на страницы курсов, названия которых сохранены в `names`, и назовите его `links`. 

In [17]:
# пример извлечения одной ссылки
courses[0].find("a").get("href")

'/2023-24/%D0%92%D1%8B%D1%81%D1%88%D0%B0%D1%8F_%D0%BC%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D0%BA%D0%B0'

In [18]:
# извлекаем все ссылки в цикле
links = [c.find("a").get("href") for c in courses]
print(links)

['/2023-24/%D0%92%D1%8B%D1%81%D1%88%D0%B0%D1%8F_%D0%BC%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D0%BA%D0%B0', '/2023-24/%D0%92%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B2_%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8%D0%BE%D0%BD%D0%BD%D1%8B%D0%B9_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7', '/2023-24/%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_%D0%B2_Python', '/2023-24/%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%B4%D0%BB%D1%8F_%D0%B2%D1%81%D0%B5%D1%85_(%D0%BE%D1%81%D0%BD%D0%BE%D0%B2%D1%8B_Python)', '/2023-24/%D0%92%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B2_%D1%82%D0%B5%D0%BE%D1%80%D0%B8%D1%8E_%D0%B2%D0%B5%D1%80%D0%BE%D1%8F%D1%82%D0%BD%D0%BE%D1%81%D1%82%D0%B5%D0%B9_%D0%B8_%D0%BC%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D1%83%D1%8E_%D1%81%D1%82%D0%B0%D1%82%D0%B8%D1%81%D1%82%D0%B8%D0%BA%D1%83', '/2023-24/%D0%9F%D1%80%D0%BE%D0%

### Задача 6

Создайте список полных кликабельных ссылок (начинающихся с `http://math-info.hse.ru`) и назовите его `full_links`.

In [19]:
# доклеиваем к каждой относительной ссылке в links
# начало http://math-info.hse.ru

full_links = ["http://math-info.hse.ru" + url for url in links]

In [20]:
# при печати все будут кликабельными

for f in full_links:
    print(f)

http://math-info.hse.ru/2023-24/%D0%92%D1%8B%D1%81%D1%88%D0%B0%D1%8F_%D0%BC%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D0%BA%D0%B0
http://math-info.hse.ru/2023-24/%D0%92%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B2_%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8%D0%BE%D0%BD%D0%BD%D1%8B%D0%B9_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7
http://math-info.hse.ru/2023-24/%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_%D0%B2_Python
http://math-info.hse.ru/2023-24/%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%B4%D0%BB%D1%8F_%D0%B2%D1%81%D0%B5%D1%85_(%D0%BE%D1%81%D0%BD%D0%BE%D0%B2%D1%8B_Python)
http://math-info.hse.ru/2023-24/%D0%92%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B2_%D1%82%D0%B5%D0%BE%D1%80%D0%B8%D1%8E_%D0%B2%D0%B5%D1%80%D0%BE%D1%8F%D1%82%D0%BD%D0%BE%D1%81%D1%82%D0%B5%D0%B9_%D0%B8_%D0%BC%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D1%83%